# 依次执行目标检测任务（patch图）

In [6]:
import os
import subprocess

# --- 0. 定义全局参数 ---
# 您想要运行的实验版本号
VERSION = "kl"

# 基础目录
BASE_MODELS_DIR = "/opt/data/private/BlackBox/models"
BASE_DATA_PATH = f"/opt/data/private/BlackBox/data/coco-patch-{VERSION}"
BASE_SAVE_DIR = f"/opt/data/private/BlackBox/save-{VERSION}/attack/detection"


# --- 1. 定义所有模型的配置 ---
# 字典存储每个模型的执行信息
# 注意：MODEL_CONFIGS 必须在这里定义
MODEL_CONFIGS = {
    "DN-DETR": {
        "dir": "DN-DETR",
        "command": [
            "python", "main.py", "-m", "dn_dab_detr",
            "--output_dir", f"{BASE_SAVE_DIR}/dn-detr/patch/",
            "--batch_size", "1",
            "--coco_path", BASE_DATA_PATH,
            "--resume", f"{BASE_MODELS_DIR}/weights/dn_detr_r50_50ep.pth",
            "--use_dn",
            "--eval",
            "--save_results",
            "--num_workers", "0"
        ]
    },
    "AnchorDETR": {
        "dir": "anchor_detr",
        "command": [
            "python", "main.py",
            "--eval",
            "--coco_path", BASE_DATA_PATH,
            "--eval_set", "test",
            "--resume", f"{BASE_MODELS_DIR}/weights/AnchorDETR_r50_c5.pth",
            "--output_dir", f"{BASE_SAVE_DIR}/anchor-detr/patch/"
        ]
    },
    "Sparse-DETR": {
        # 注意：Sparse-DETR 使用 bash 脚本，需要特殊处理
        "dir": "sparse_detr",
        "command": [
            "bash", "./configs/r50_sparse_detr_rho_0.1.sh",
            "--resume", f"{BASE_MODELS_DIR}/weights/sparse_detr_r50_10.pth",
            "--eval",
            "--coco_path", BASE_DATA_PATH,
            "--output_dir", f"{BASE_SAVE_DIR}/sparse-detr/patch"
        ]
    },
    "Deformable-DETR": {
        "dir": "Deformable-DETR",
        "command": [
            "python", "main.py",
            "--eval",
            "--resume", f"{BASE_MODELS_DIR}/weights/r50_deformable_detr_single_scale-checkpoint.pth",
            "--output_dir", f"{BASE_SAVE_DIR}/deformable-detr/patch",
            "--coco_path", BASE_DATA_PATH,
            "--num_feature_levels", "1"
        ]
    },
    "DETR": {
        "dir": "detr",
        "command": [
            "python", "main.py",
            "--batch_size", "2",
            "--no_aux_loss",
            "--eval",
            "--resume", "https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth",
            "--coco_path", BASE_DATA_PATH,
            "--output_dir", f"{BASE_SAVE_DIR}/detr/patch"
        ]
    }
}

# --- 2. 在定义 MODEL_CONFIGS 之后再打印 ---
print(f"--- 准备运行 {len(MODEL_CONFIGS)} 个模型，使用数据版本: {VERSION} ---")


# --- 3. 循环执行所有模型的评估 ---
# 保存初始工作目录，以便执行完毕后返回
original_cwd = os.getcwd()

try:
    for model_name, config in MODEL_CONFIGS.items():
        model_dir = os.path.join(BASE_MODELS_DIR, config["dir"])
        
        print("-" * 50)
        print(f"开始执行模型: {model_name}")
        print(f"切换到目录: {model_dir}")
        
        # 切换工作目录 (对应于 'cd ...')
        os.chdir(model_dir)
        
        # 确保输出目录存在
        # output_dir是command list中的一个参数，我们手动提取第一个作为检查对象
        # 查找 --output_dir 参数的位置，并获取其后的路径
        try:
            output_dir_param_index = config["command"].index("--output_dir")
            output_dir_path = config["command"][output_dir_param_index + 1]
            os.makedirs(output_dir_path, exist_ok=True)
            print(f"输出结果将保存到: {output_dir_path}")
        except ValueError:
            # 如果没有 --output_dir 参数，则跳过目录创建
            pass


        # 执行命令 (对应于 'python main.py ...' 或 'bash ...')
        # 打印执行的命令（易于调试）
        print(f"执行命令: {' '.join(config['command'])}")
        
        # subprocess.run 运行命令，check=True 表示如果命令返回非零退出码（通常表示错误），则抛出异常
        # 这里为了简化Jupyter输出，暂时不捕获大量输出，让它直接流向Notebook
        subprocess.run(config["command"], check=True)
        
        print(f"{model_name} 执行成功。")


finally:
    # 无论脚本是否出错，都确保回到初始工作目录
    os.chdir(original_cwd)
    print("-" * 50)
    print("所有模型评估流程已结束。工作目录已恢复。")

--- 准备运行 5 个模型，使用数据版本: kl ---
--------------------------------------------------
开始执行模型: DN-DETR
切换到目录: /opt/data/private/BlackBox/models/DN-DETR
输出结果将保存到: /opt/data/private/BlackBox/save-kl/attack/detection/dn-detr/patch/
执行命令: python main.py -m dn_dab_detr --output_dir /opt/data/private/BlackBox/save-kl/attack/detection/dn-detr/patch/ --batch_size 1 --coco_path /opt/data/private/BlackBox/data/coco-patch-kl --resume /opt/data/private/BlackBox/models/weights/dn_detr_r50_50ep.pth --use_dn --eval --save_results --num_workers 0
Not using distributed mode
[10/29 17:37:18.203]: git:
  sha: ff3902a20d521ead052d1243ff249b19bc1ce531, status: has uncommited changes, branch: main

[10/29 17:37:18.204]: Command: main.py -m dn_dab_detr --output_dir /opt/data/private/BlackBox/save-kl/attack/detection/dn-detr/patch/ --batch_size 1 --coco_path /opt/data/private/BlackBox/data/coco-patch-kl --resume /opt/data/private/BlackBox/models/weights/dn_detr_r50_50ep.pth --use_dn --eval --save_results --num_work

/usr/local/lib/python3.8/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
main.py:313: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default val

DN-DETR 执行成功。
--------------------------------------------------
开始执行模型: AnchorDETR
切换到目录: /opt/data/private/BlackBox/models/anchor_detr
输出结果将保存到: /opt/data/private/BlackBox/save-kl/attack/detection/anchor-detr/patch/
执行命令: python main.py --eval --coco_path /opt/data/private/BlackBox/data/coco-patch-kl --eval_set test --resume /opt/data/private/BlackBox/models/weights/AnchorDETR_r50_c5.pth --output_dir /opt/data/private/BlackBox/save-kl/attack/detection/anchor-detr/patch/
Not using distributed mode
git:
  sha: 0b4358a1caacf4b311b8653312e145f1c94de3d3, status: clean, branch: main

Namespace(attention_type='RCDA', auto_resume=False, aux_loss=True, backbone='resnet50', batch_size=1, bbox_loss_coef=5, cache_mode=False, clip_max_norm=0.1, cls_loss_coef=2, coco_panoptic_path=None, coco_path='/opt/data/private/BlackBox/data/coco-patch-kl', dataset_file='coco', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=True, dim_feedforward=1024, distributed=False, dropout=0.0, enc_layers=6, epoc

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
main.py:246: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release,

Test:  [  0/288]  eta: 0:07:29  class_error: 0.00  loss: 29.2536 (29.2536)  time: 1.5612  data: 0.2681  max mem: 585
Test:  [100/288]  eta: 0:00:15  class_error: 0.00  loss: 12.8876 (13.1693)  time: 0.0625  data: 0.0016  max mem: 666
Test:  [200/288]  eta: 0:00:06  class_error: 0.00  loss: 12.5825 (14.1864)  time: 0.0628  data: 0.0016  max mem: 666
Test:  [287/288]  eta: 0:00:00  class_error: 0.00  loss: 14.1813 (15.1251)  time: 0.0593  data: 0.0017  max mem: 666
Test: Total time: 0:00:19 (0.0693 s / it)
Averaged stats: class_error: 0.00  loss: 14.1813 (15.1251)
Accumulating evaluation results...
DONE (t=0.06s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.565
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.885
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.627
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 

...

[DEBUG][engine] batch detections per image = [100, 100], batch_total=200
[DEBUG][engine] first image_id=1, keys=['scores', 'labels', 'boxes'], n_scores=100
[DEBUG][export] wrote 200 detections for batch to /opt/data/private/BlackBox/save-kl/attack/detection/deformable-detr/patch/detections_stream.jsonl
[DEBUG][coco_eval] prepare(iou=bbox) -> 200 rows
[DEBUG][coco_eval] first row: {'image_id': 1, 'category_id': 1, 'bbox': [399.47906494140625, 217.9432830810547, 181.86688232421875, 488.76361083984375], 'score': 0.8650018572807312}
Test:  [  0/144]  eta: 0:03:50  class_error: 0.00  loss: 15.9853 (15.9853)  loss_ce: 1.8764 (1.8764)  loss_bbox: 0.3503 (0.3503)  loss_giou: 0.4143 (0.4143)  loss_ce_0: 1.6130 (1.6130)  loss_bbox_0: 0.5274 (0.5274)  loss_giou_0: 0.5257 (0.5257)  loss_ce_1: 1.6838 (1.6838)  loss_bbox_1: 0.3780 (0.3780)  loss_giou_1: 0.4281 (0.4281)  loss_ce_2: 1.8509 (1.8509)  loss_bbox_2: 0.3471 (0.3471)  loss_giou_2: 0.4183 (0.4183)  loss_ce_3: 2.0578 (2.0578)  loss_bbox_3: 0.

/usr/local/lib/python3.8/dist-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Test:  [140/144]  eta: 0:00:00  class_error: 0.00  loss: 13.0312 (14.7029)  loss_ce: 1.7093 (1.8606)  loss_bbox: 0.2195 (0.2866)  loss_giou: 0.3468 (0.3454)  loss_ce_0: 1.1639 (1.6521)  loss_bbox_0: 0.3599 (0.3780)  loss_giou_0: 0.4079 (0.4069)  loss_ce_1: 1.5284 (1.6956)  loss_bbox_1: 0.2161 (0.3210)  loss_giou_1: 0.3670 (0.3631)  loss_ce_2: 1.7161 (1.7491)  loss_bbox_2: 0.2270 (0.3131)  loss_giou_2: 0.3530 (0.3613)  loss_ce_3: 1.7221 (1.8309)  loss_bbox_3: 0.2292 (0.2962)  loss_giou_3: 0.3283 (0.3531)  loss_ce_4: 1.6806 (1.8605)  loss_bbox_4: 0.2247 (0.2842)  loss_giou_4: 0.3408 (0.3453)  loss_ce_unscaled: 0.8546 (0.9303)  class_error_unscaled: 0.0000 (0.0000)  loss_bbox_unscaled: 0.0439 (0.0573)  loss_giou_unscaled: 0.1734 (0.1727)  cardinality_error_unscaled: 298.5000 (297.9220)  loss_ce_0_unscaled: 0.5819 (0.8260)  loss_bbox_0_unscaled: 0.0720 (0.0756)  loss_giou_0_unscaled: 0.2039 (0.2034)  cardinality_error_0_unscaled: 298.5000 (297.9078)  loss_ce_1_unscaled: 0.7642 (0.8478)  lo

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test:  [  0/144]  eta: 0:03:11  loss: 1.3240 (1.3240)  loss_ce: 0.2478 (0.2478)  loss_bbox: 0.5027 (0.5027)  loss_giou: 0.5735 (0.5735)  loss_ce_unscaled: 0.2478 (0.2478)  class_error_unscaled: 0.0000 (0.0000)  loss_bbox_unscaled: 0.1005 (0.1005)  loss_giou_unscaled: 0.2868 (0.2868)  cardinality_error_unscaled: 8.5000 (8.5000)  class_error: 0.0000 (0.0000)  time: 1.3315  data: 0.2707  max mem: 640
Test:  [ 10/144]  eta: 0:00:25  loss: 0.7481 (0.8328)  loss_ce: 0.3319 (0.3366)  loss_bbox: 0.1611 (0.1831)  loss_giou: 0.2226 (0.3132)  loss_ce_unscaled: 0.3319 (0.3366)  class_error_unscaled: 0.0000 (0.0000)  loss_bbox_unscaled: 0.0322 (0.0366)  loss_giou_unscaled: 0.1113 (0.1566)  cardinality_error_unscaled: 12.5000 (13.2273)  class_error: 0.0000 (0.0000)  time: 0.1902  data: 0.0332  max mem: 790
Test:  [ 20/144]  eta: 0:00:15  loss: 0.7861 (0.8609)  loss_ce: 0.2794 (0.3316)  loss_bbox: 0.1671 (0.2092)  loss_giou: 0.2506 (0.3201)  loss_ce_unscaled: 0.2794 (0.3316)  class_error_unscaled: 0.

# 针对clean图做目标检测：

In [1]:
import os
import subprocess

# --- 0. 定义全局参数 ---
# 您想要运行的实验版本号
VERSION = "clean"

# 基础目录
BASE_MODELS_DIR = "/opt/data/private/BlackBox/models"
BASE_DATA_PATH = "/opt/data/private/BlackBox/data/coco"
BASE_SAVE_DIR = "/opt/data/private/BlackBox/save/attack/detection"


# --- 1. 定义所有模型的配置 ---
# 字典存储每个模型的执行信息
# 注意：MODEL_CONFIGS 必须在这里定义
MODEL_CONFIGS = {
    "DN-DETR": {
        "dir": "DN-DETR",
        "command": [
            "python", "main.py", "-m", "dn_dab_detr",
            "--output_dir", f"{BASE_SAVE_DIR}/dn-detr/ori/",
            "--batch_size", "1",
            "--coco_path", BASE_DATA_PATH,
            "--resume", f"{BASE_MODELS_DIR}/weights/dn_detr_r50_50ep.pth",
            "--use_dn",
            "--eval",
            "--save_results",
            "--num_workers", "0"
        ]
    },
    "AnchorDETR": {
        "dir": "anchor_detr",
        "command": [
            "python", "main.py",
            "--eval",
            "--coco_path", BASE_DATA_PATH,
            "--eval_set", "test",
            "--resume", f"{BASE_MODELS_DIR}/weights/AnchorDETR_r50_c5.pth",
            "--output_dir", f"{BASE_SAVE_DIR}/anchor-detr/ori/"
        ]
    },
    "Sparse-DETR": {
        # 注意：Sparse-DETR 使用 bash 脚本，需要特殊处理
        "dir": "sparse_detr",
        "command": [
            "bash", "./configs/r50_sparse_detr_rho_0.1.sh",
            "--resume", f"{BASE_MODELS_DIR}/weights/sparse_detr_r50_10.pth",
            "--eval",
            "--coco_path", BASE_DATA_PATH,
            "--output_dir", f"{BASE_SAVE_DIR}/sparse-detr/ori"
        ]
    },
    "Deformable-DETR": {
        "dir": "Deformable-DETR",
        "command": [
            "python", "main.py",
            "--eval",
            "--resume", f"{BASE_MODELS_DIR}/weights/r50_deformable_detr_single_scale-checkpoint.pth",
            "--output_dir", f"{BASE_SAVE_DIR}/deformable-detr/ori",
            "--coco_path", BASE_DATA_PATH,
            "--num_feature_levels", "1"
        ]
    },
    "DETR": {
        "dir": "detr",
        "command": [
            "python", "main.py",
            "--batch_size", "2",
            "--no_aux_loss",
            "--eval",
            "--resume", "https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth",
            "--coco_path", BASE_DATA_PATH,
            "--output_dir", f"{BASE_SAVE_DIR}/detr/ori"
        ]
    }
}

# --- 2. 在定义 MODEL_CONFIGS 之后再打印 ---
print(f"--- 准备运行 {len(MODEL_CONFIGS)} 个模型，使用数据版本: {VERSION} ---")


# --- 3. 循环执行所有模型的评估 ---
# 保存初始工作目录，以便执行完毕后返回
original_cwd = os.getcwd()

try:
    for model_name, config in MODEL_CONFIGS.items():
        model_dir = os.path.join(BASE_MODELS_DIR, config["dir"])
        
        print("-" * 50)
        print(f"开始执行模型: {model_name}")
        print(f"切换到目录: {model_dir}")
        
        # 切换工作目录 (对应于 'cd ...')
        os.chdir(model_dir)
        
        # 确保输出目录存在
        # output_dir是command list中的一个参数，我们手动提取第一个作为检查对象
        # 查找 --output_dir 参数的位置，并获取其后的路径
        try:
            output_dir_param_index = config["command"].index("--output_dir")
            output_dir_path = config["command"][output_dir_param_index + 1]
            os.makedirs(output_dir_path, exist_ok=True)
            print(f"输出结果将保存到: {output_dir_path}")
        except ValueError:
            # 如果没有 --output_dir 参数，则跳过目录创建
            pass


        # 执行命令 (对应于 'python main.py ...' 或 'bash ...')
        # 打印执行的命令（易于调试）
        print(f"执行命令: {' '.join(config['command'])}")
        
        # subprocess.run 运行命令，check=True 表示如果命令返回非零退出码（通常表示错误），则抛出异常
        # 这里为了简化Jupyter输出，暂时不捕获大量输出，让它直接流向Notebook
        subprocess.run(config["command"], check=True)
        
        print(f"{model_name} 执行成功。")


finally:
    # 无论脚本是否出错，都确保回到初始工作目录
    os.chdir(original_cwd)
    print("-" * 50)
    print("所有模型评估流程已结束。工作目录已恢复。")

--- 准备运行 5 个模型，使用数据版本: clean ---
--------------------------------------------------
开始执行模型: DN-DETR
切换到目录: /opt/data/private/BlackBox/models/DN-DETR
输出结果将保存到: /opt/data/private/BlackBox/save/attack/detection/dn-detr/ori/
执行命令: python main.py -m dn_dab_detr --output_dir /opt/data/private/BlackBox/save/attack/detection/dn-detr/ori/ --batch_size 1 --coco_path /opt/data/private/BlackBox/data/coco --resume /opt/data/private/BlackBox/models/weights/dn_detr_r50_50ep.pth --use_dn --eval --save_results --num_workers 0
Not using distributed mode
[10/30 20:24:20.657]: git:
  sha: ff3902a20d521ead052d1243ff249b19bc1ce531, status: has uncommited changes, branch: main

[10/30 20:24:20.658]: Command: main.py -m dn_dab_detr --output_dir /opt/data/private/BlackBox/save/attack/detection/dn-detr/ori/ --batch_size 1 --coco_path /opt/data/private/BlackBox/data/coco --resume /opt/data/private/BlackBox/models/weights/dn_detr_r50_50ep.pth --use_dn --eval --save_results --num_workers 0
[10/30 20:24:20.659]: Fu

/usr/local/lib/python3.8/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
main.py:313: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default val

DN-DETR 执行成功。
--------------------------------------------------
开始执行模型: AnchorDETR
切换到目录: /opt/data/private/BlackBox/models/anchor_detr
输出结果将保存到: /opt/data/private/BlackBox/save/attack/detection/anchor-detr/ori/
执行命令: python main.py --eval --coco_path /opt/data/private/BlackBox/data/coco --eval_set test --resume /opt/data/private/BlackBox/models/weights/AnchorDETR_r50_c5.pth --output_dir /opt/data/private/BlackBox/save/attack/detection/anchor-detr/ori/
Not using distributed mode
git:
  sha: 0b4358a1caacf4b311b8653312e145f1c94de3d3, status: has uncommited changes, branch: main

Namespace(attention_type='RCDA', auto_resume=False, aux_loss=True, backbone='resnet50', batch_size=1, bbox_loss_coef=5, cache_mode=False, clip_max_norm=0.1, cls_loss_coef=2, coco_panoptic_path=None, coco_path='/opt/data/private/BlackBox/data/coco', dataset_file='coco', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=True, dim_feedforward=1024, distributed=False, dropout=0.0, enc_layers=6, epochs=50, eval

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
main.py:246: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release,

Test:  [  0/288]  eta: 0:07:42  class_error: 0.00  loss: 27.2532 (27.2532)  time: 1.6046  data: 0.3061  max mem: 585
Test:  [100/288]  eta: 0:00:19  class_error: 0.00  loss: 13.3641 (12.9082)  time: 0.0789  data: 0.0020  max mem: 666
Test:  [200/288]  eta: 0:00:08  class_error: 0.00  loss: 12.1908 (13.4187)  time: 0.0824  data: 0.0040  max mem: 666
Test:  [287/288]  eta: 0:00:00  class_error: 0.00  loss: 11.7374 (14.3278)  time: 0.0769  data: 0.0020  max mem: 666
Test: Total time: 0:00:25 (0.0877 s / it)
Averaged stats: class_error: 0.00  loss: 11.7374 (14.3278)
Accumulating evaluation results...
DONE (t=0.06s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.595
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.912
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.666
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 

...

[DEBUG][engine] batch detections per image = [100, 100], batch_total=200
[DEBUG][engine] first image_id=1, keys=['scores', 'labels', 'boxes'], n_scores=100
[DEBUG][export] wrote 200 detections for batch to /opt/data/private/BlackBox/save/attack/detection/deformable-detr/ori/detections_stream.jsonl
[DEBUG][coco_eval] prepare(iou=bbox) -> 200 rows
[DEBUG][coco_eval] first row: {'image_id': 1, 'category_id': 1, 'bbox': [405.9403076171875, 223.2398681640625, 170.6689453125, 476.369873046875], 'score': 0.8941296339035034}
Test:  [  0/144]  eta: 0:06:27  class_error: 0.00  loss: 14.0389 (14.0389)  loss_ce: 1.7694 (1.7694)  loss_bbox: 0.3215 (0.3215)  loss_giou: 0.3755 (0.3755)  loss_ce_0: 1.3795 (1.3795)  loss_bbox_0: 0.3344 (0.3344)  loss_giou_0: 0.3921 (0.3921)  loss_ce_1: 1.4879 (1.4879)  loss_bbox_1: 0.2943 (0.2943)  loss_giou_1: 0.3744 (0.3744)  loss_ce_2: 1.6065 (1.6065)  loss_bbox_2: 0.3223 (0.3223)  loss_giou_2: 0.3755 (0.3755)  loss_ce_3: 1.7908 (1.7908)  loss_bbox_3: 0.3064 (0.3064

/usr/local/lib/python3.8/dist-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Test:  [140/144]  eta: 0:00:00  class_error: 0.00  loss: 12.7382 (12.4241)  loss_ce: 1.5588 (1.6103)  loss_bbox: 0.2211 (0.2534)  loss_giou: 0.3228 (0.3164)  loss_ce_0: 1.0602 (1.2984)  loss_bbox_0: 0.2642 (0.2915)  loss_giou_0: 0.3055 (0.3419)  loss_ce_1: 1.4437 (1.3637)  loss_bbox_1: 0.2276 (0.2716)  loss_giou_1: 0.2892 (0.3283)  loss_ce_2: 1.5591 (1.4879)  loss_bbox_2: 0.2234 (0.2621)  loss_giou_2: 0.3043 (0.3217)  loss_ce_3: 1.5999 (1.5463)  loss_bbox_3: 0.2123 (0.2569)  loss_giou_3: 0.3250 (0.3191)  loss_ce_4: 1.5381 (1.5804)  loss_bbox_4: 0.2314 (0.2563)  loss_giou_4: 0.3212 (0.3180)  loss_ce_unscaled: 0.7794 (0.8051)  class_error_unscaled: 0.0000 (0.0000)  loss_bbox_unscaled: 0.0442 (0.0507)  loss_giou_unscaled: 0.1614 (0.1582)  cardinality_error_unscaled: 298.5000 (297.9113)  loss_ce_0_unscaled: 0.5301 (0.6492)  loss_bbox_0_unscaled: 0.0528 (0.0583)  loss_giou_0_unscaled: 0.1527 (0.1709)  cardinality_error_0_unscaled: 298.5000 (297.9078)  loss_ce_1_unscaled: 0.7219 (0.6818)  lo

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test:  [  0/144]  eta: 0:03:46  loss: 1.3419 (1.3419)  loss_ce: 0.1716 (0.1716)  loss_bbox: 0.5507 (0.5507)  loss_giou: 0.6196 (0.6196)  loss_ce_unscaled: 0.1716 (0.1716)  class_error_unscaled: 0.0000 (0.0000)  loss_bbox_unscaled: 0.1101 (0.1101)  loss_giou_unscaled: 0.3098 (0.3098)  cardinality_error_unscaled: 4.0000 (4.0000)  class_error: 0.0000 (0.0000)  time: 1.5758  data: 0.4350  max mem: 640
Test:  [ 10/144]  eta: 0:00:30  loss: 0.7214 (0.7729)  loss_ce: 0.2161 (0.2832)  loss_bbox: 0.1462 (0.1831)  loss_giou: 0.2254 (0.3067)  loss_ce_unscaled: 0.2161 (0.2832)  class_error_unscaled: 0.0000 (0.0000)  loss_bbox_unscaled: 0.0292 (0.0366)  loss_giou_unscaled: 0.1127 (0.1533)  cardinality_error_unscaled: 7.5000 (10.7727)  class_error: 0.0000 (0.0000)  time: 0.2311  data: 0.0477  max mem: 790
Test:  [ 20/144]  eta: 0:00:19  loss: 0.7561 (0.8033)  loss_ce: 0.2728 (0.2815)  loss_bbox: 0.1663 (0.2061)  loss_giou: 0.2532 (0.3157)  loss_ce_unscaled: 0.2728 (0.2815)  class_error_unscaled: 0.0